<a href="https://colab.research.google.com/github/SnkhchyanV/NLP/blob/main/Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>